## Adiabatic Quantum Computing workflow for Computational Materials Chemistry


In [3]:
from pymatgen.core.structure import Molecule, Structure, Lattice
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.ext.matproj import MPRester
from pymatgen.io.ase import AseAtomsAdaptor

from ase.visualize import view
import copy
import time
import itertools

import numpy as np

The **objectives** are:
- maximising the number of bonds

$$
E_{bonds} = - p \sum_{i}^{N_{atoms}}\sum_{j>i}^{N_{atoms}} A_{i,j} x_{i} x_{j} 
$$

where $A_{i,j} = 1$ if atoms i and j are bonded and $A_{i,j} = 0$ otherwise.

- minimising the number of undercoordinated atoms

$$
E_{coord} = - q \sum_{i}^{N_{atoms}} \sum_{j}^{N^{i}_{neigh}}\sum_{k>j}^{N^{i}_{neigh}}  x_{i} x_{j} 
$$

Where $N^{i}_{bonds}$ is the number of neighbours the atom i has in absence of vacancies.

The scaling factors p and q in the above expressions can be use to determine to weight of the bonds/coordination into the final energy.\

The **constraints** are defined in terms of number of vacancies:

$$
\sum_{i}^{N_{atoms}} x_{i} = N_{atoms} - N_{vacancies} 
$$

that can be expressed as:

$$
\sum_{i}^{N_{atoms}} x_{i} - (N_{atoms} - N_{vacancies}) = 0 
$$

# The functions

In [4]:
def build_adjacency_matrix(structure):
    # structure = pymatgen Structure object
    
    import numpy as np
    
    distance_matrix_pbc = np.round(structure.distance_matrix,5)

    shells = np.unique(distance_matrix_pbc[0])

    adjacency_matrix = np.round(distance_matrix_pbc,5) == np.round(shells[1],5)
    adjacency_matrix = adjacency_matrix.astype(int)
    
    return adjacency_matrix

In [60]:
def build_quadratic_model(structure,use_coord = True, num_vacancies = 0, 
                          weight_1=10, weight_2 = 1, lagrange = 1000):
    # structure = pymatgen Structure object
    
    from dimod import BinaryQuadraticModel
    
    X = np.arange(structure.num_sites)
    
    adjacency_matrix = build_adjacency_matrix(structure)

    Q = np.triu(-weight_1*adjacency_matrix.astype(int),0)
    
    bqm = BinaryQuadraticModel.from_qubo(Q)
    
    if use_coord == True:
        for i in range(structure.num_sites):  
            neighbours = np.where(adjacency_matrix[i,:] == 1)[0]
            for j in range(len(neighbours)):
                for k in range(len(neighbours)):
                    if k > j:
                        bqm.add_interaction(X[neighbours[j]],X[neighbours[k]],1)
    

    if num_vacancies == 0:
        print('Unconstrained quadratic model used')
        
        return bqm
    elif num_vacancies > 0:
        print('Unconstrained quadratic model + contraints used')
        
        c_n_vacancies = [(i,1) for i in X]

        bqm.add_linear_equality_constraint(
                c_n_vacancies,
                constant= -(structure.num_sites-num_vacancies),
                lagrange_multiplier = lagrange
                )

        return bqm
    
    else:
        print('Please select a positive integer number of vacancies')
     
        return None

In [26]:
def run_anneal(bqm,num_reads = 1000, label='Test anneal', dataframe = True, remove_broken_chains = True,
              return_config_E = False):
    
    from dwave.system import EmbeddingComposite, DWaveSampler
    
    sampler = EmbeddingComposite(DWaveSampler())
    
    result = sampler.sample(bqm, num_reads = num_reads,  label=label)
    
    if dataframe == True:
        result_df = result.to_pandas_dataframe()
        
        if remove_broken_chains == True:
            result_df = result_df[result_df['chain_break_fraction'] == 0.]
            
            if return_config_E == True:
                n_atoms = len(bqm.to_numpy_vectors()[0])
                
                config = result_df.iloc[:,0:n_atoms].to_numpy()
                energies = result_df['energy'].to_numpy()
                
                return result_df, config, energies
            else:
                return result_df
        else:
            return result_df
    else:
        return result

In [73]:
def find_unique_E_structures(energies,configurations,min_energy = 0):
    np.unique(energy,return_index=True)
    below_min_energy = np.where(np.unique(energy,return_index=True)[0] < 0.)[0]
    return np.unique(energy,return_index=True)[0][below_min_energy], np.unique(energy,return_index=True)[1][below_min_energy]

In [84]:
def display_low_E_structures(structure,energies,configurations, min_energy = 0):
    
    low_energy, low_config = find_unique_E_structures(energies,configurations,min_energy = min_energy)
    
    for i in low_config:
        structure_2 = copy.deepcopy(structure)
        for j in np.where(configurations[i] == 0)[0]:
            structure_2.replace(j,1)
        
        view(AseAtomsAdaptor().get_atoms(structure_2))
    

# Graphene

## Build structure

In [58]:
lattice = np.array([[ 1.233862, -2.137112,  0.      ],
                   [ 1.233862,  2.137112,  0.      ],
                   [ 0.      ,  0.      ,  8.685038]])

graphene = Structure(lattice, species=['C','C'], coords=[[2/3, 1/3, 0. ],[1/3, 2/3, 0.]])
graphene = SpacegroupAnalyzer(graphene).get_conventional_standard_structure()

n_supercell = 3
scaling_matrix = np.identity(3)*n_supercell
scaling_matrix[2][2] = 1
graphene_supercell = copy.deepcopy(graphene)
graphene_supercell.make_supercell(scaling_matrix)
#graphene_supercell

In [61]:
bqm = build_quadratic_model(graphene_supercell,num_vacancies=2)
dataframe, config, energy = run_anneal(bqm,num_reads=100, return_config_E=True)

Unconstrained quadratic model + contraints used


In [74]:
find_unique_E_structures(energy,config)

(array([-178., -168., -167.]), array([ 0,  5, 16]))

In [88]:
np.unique(energy,return_index=True)

(array([-178., -168., -167.,  808.,  837.,  846.,  847.,  856.,  857.,
         858.,  859., 3873., 3885., 8887.]),
 array([ 0,  5, 16, 27, 28, 32, 35, 52, 56, 74, 78, 80, 81, 82]))

In [86]:
display_low_E_structures(graphene_supercell,energy, config)

2022-06-24 16:17:30.525 python[15309:204905] *** CFMessagePort: bootstrap_register(): failed 1100 (0x44c) 'Permission denied', port = 0x14c07, name = 'python.ServiceProvider'
See /usr/include/servers/bootstrap_defs.h for the error codes.


In [87]:
#view(AseAtomsAdaptor().get_atoms(graphene_supercell),)

# Copper

## Build structure

In [55]:
lattice = np.array([[0.      , 1.810631, 1.810631],
                   [1.810631, 0.      , 1.810631],
                   [1.810631, 1.810631, 0.      ]])

copper = Structure(lattice, species=['Cu'], coords=[[0., 0., 0. ]])

copper = SpacegroupAnalyzer(copper).get_conventional_standard_structure()

n_supercell = 2
scaling_matrix = np.identity(3)*n_supercell
copper_supercell = copy.deepcopy(copper)
copper_supercell.make_supercell(scaling_matrix)

In [45]:
view(AseAtomsAdaptor().get_atoms(copper_supercell))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/qc/bin/p...>

In [41]:
bqm = build_quadratic_model(copper_supercell,num_vacancies=1, weight_1=1)
#dataframe, config, energy = run_anneal(bqm,num_reads=100, return_config_E=True)

[[0. 8. 8. ... 4. 3. 4.]
 [0. 0. 4. ... 3. 4. 3.]
 [0. 0. 0. ... 4. 3. 4.]
 ...
 [0. 0. 0. ... 0. 4. 8.]
 [0. 0. 0. ... 0. 0. 8.]
 [0. 0. 0. ... 0. 0. 0.]]
Unconstrained quadratic model + contraints used
[[-61000.   2008.   2008. ...   2004.   2003.   2004.]
 [     0. -61000.   2004. ...   2003.   2004.   2003.]
 [     0.      0. -61000. ...   2004.   2003.   2004.]
 ...
 [     0.      0.      0. ... -61000.   2004.   2008.]
 [     0.      0.      0. ...      0. -61000.   2008.]
 [     0.      0.      0. ...      0.      0. -61000.]]


/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_13721/4185597591.py:23: DeprecationWarning: bqm.to_numpy_matrix() is deprecated since dimod 0.10.0 and will be removed in 0.12.0
  print(bqm.to_numpy_matrix())
/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_13721/4185597591.py:38: DeprecationWarning: bqm.to_numpy_matrix() is deprecated since dimod 0.10.0 and will be removed in 0.12.0
  print(bqm.to_numpy_matrix())


In [43]:
build_adjacency_matrix(copper_supercell)[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0])

In [52]:
distance_matrix_pbc = np.round(copper_supercell.distance_matrix,5)

shells = np.unique(distance_matrix_pbc[0])

adjacency_matrix = np.round(distance_matrix_pbc,5) == np.round(shells[1],5)
adjacency_matrix = adjacency_matrix.astype(int)
np.sum(adjacency_matrix,axis=1)

array([12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12])

In [57]:
np.round(copper_supercell.distance_matrix,5)[0]

array([0.     , 3.62126, 3.62126, 5.12124, 3.62126, 5.12124, 5.12124,
       6.27221, 2.56062, 2.56062, 2.56062, 2.56062, 4.43512, 4.43512,
       4.43512, 4.43512, 2.56062, 2.56062, 4.43512, 4.43512, 2.56062,
       2.56062, 4.43512, 4.43512, 2.56062, 4.43512, 2.56062, 4.43512,
       2.56062, 4.43512, 2.56062, 4.43512])